# MLflow + Visualizaciones: Matplotlib y Seaborn

## Objetivos
- Integrar visualizaciones con MLflow
- Guardar gráficos como artifacts
- Crear dashboards interactivos
- Best practices para visualización en ML

## Tipos de Visualizaciones
1. **Métricas de entrenamiento** - Loss curves, accuracy
2. **Evaluación de modelos** - Confusion matrix, ROC curves
3. **Feature importance** - Importancia de variables
4. **Datos** - Distribuciones, correlaciones
5. **Predicciones** - Actual vs Predicted

## 1. Setup e Importaciones

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Core
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    confusion_matrix, classification_report, roc_curve, auc,
    precision_recall_curve, accuracy_score, f1_score
)

# Visualización
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# MLflow
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Importaciones completadas')

## 2. Configurar MLflow

In [ ]:
# Configurar MLflow
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment('visualization-examples')

# Crear cliente
client = MlflowClient()

print(f'✓ MLflow URI: {mlflow.get_tracking_uri()}')
print(f'✓ Experiment: visualization-examples')

## 3. Generar Datos de Ejemplo

In [ ]:
# Dataset de clasificación
X, y = make_classification(
    n_samples=2000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    n_classes=2,
    random_state=42
)

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Crear DataFrame para visualizaciones
feature_names = [f'feature_{i}' for i in range(X.shape[1])]
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

print(f'✓ Dataset: {X.shape[0]} muestras, {X.shape[1]} features')
print(f'✓ Clases: {np.unique(y, return_counts=True)}')

## 4. Visualización 1: Distribución de Datos con Matplotlib

In [ ]:
def plot_data_distribution(df, save_path='data_distribution.png'):
    """Visualizar distribución de features"""
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('Distribución de Features', fontsize=16, fontweight='bold', y=1.02)
    
    # Seleccionar 6 features para visualizar
    features_to_plot = feature_names[:6]
    
    for idx, (ax, feature) in enumerate(zip(axes.flat, features_to_plot)):
        # Histograma por clase
        for class_label in [0, 1]:
            data_class = df[df['target'] == class_label][feature]
            ax.hist(data_class, bins=30, alpha=0.6, label=f'Class {class_label}')
        
        ax.set_xlabel(feature, fontsize=10)
        ax.set_ylabel('Frecuencia', fontsize=10)
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    return save_path

# Crear y guardar gráfico
with mlflow.start_run(run_name='data-exploration'):
    # Crear visualización
    plot_path = plot_data_distribution(df)
    
    # Guardar en MLflow
    mlflow.log_artifact(plot_path, artifact_path='plots')
    
    # Log estadísticas
    mlflow.log_params({
        'n_samples': len(df),
        'n_features': len(feature_names),
        'n_classes': len(np.unique(y))
    })
    
    print('✓ Gráfico guardado en MLflow')

## 5. Visualización 2: Matriz de Correlación con Seaborn

In [ ]:
def plot_correlation_matrix(df, save_path='correlation_matrix.png'):
    """Visualizar matriz de correlación con seaborn"""
    
    # Calcular correlación (solo features numéricas)
    corr_matrix = df[feature_names[:10]].corr()  # Primeras 10 features
    
    # Crear figura
    plt.figure(figsize=(12, 10))
    
    # Heatmap con seaborn
    sns.heatmap(
        corr_matrix,
        annot=True,
        fmt='.2f',
        cmap='coolwarm',
        center=0,
        square=True,
        linewidths=1,
        cbar_kws={'shrink': 0.8}
    )
    
    plt.title('Matriz de Correlación de Features', fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    return save_path

# Crear y guardar
with mlflow.start_run(run_name='correlation-analysis'):
    plot_path = plot_correlation_matrix(df)
    mlflow.log_artifact(plot_path, artifact_path='plots')
    
    # Log correlaciones más altas
    corr_matrix = df[feature_names[:10]].corr()
    upper_triangle = np.triu(corr_matrix, k=1)
    max_corr = np.max(np.abs(upper_triangle))
    mlflow.log_metric('max_correlation', max_corr)
    
    print(f'✓ Correlación máxima: {max_corr:.3f}')

## 6. Entrenar Modelo para Visualizaciones

In [ ]:
# Entrenar Random Forest
with mlflow.start_run(run_name='model-training') as run:
    # Modelo
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42
    )
    
    # Entrenar
    model.fit(X_train, y_train)
    
    # Predicciones
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    mlflow.log_params({
        'n_estimators': 100,
        'max_depth': 10
    })
    
    mlflow.log_metrics({
        'accuracy': accuracy,
        'f1_score': f1
    })
    
    mlflow.sklearn.log_model(model, 'random-forest')
    
    run_id = run.info.run_id
    
    print(f'✓ Modelo entrenado')
    print(f'  Accuracy: {accuracy:.4f}')
    print(f'  F1-Score: {f1:.4f}')

## 7. Visualización 3: Confusion Matrix

In [ ]:
def plot_confusion_matrix(y_true, y_pred, save_path='confusion_matrix.png'):
    """Matriz de confusión mejorada con seaborn"""
    
    # Calcular matriz
    cm = confusion_matrix(y_true, y_pred)
    
    # Normalizar
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Crear figura con dos subplots
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Matriz absoluta
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        ax=axes[0],
        cbar_kws={'label': 'Count'}
    )
    axes[0].set_title('Confusion Matrix (Counts)', fontweight='bold')
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('Actual')
    
    # Matriz normalizada
    sns.heatmap(
        cm_normalized,
        annot=True,
        fmt='.2%',
        cmap='Greens',
        ax=axes[1],
        cbar_kws={'label': 'Percentage'}
    )
    axes[1].set_title('Confusion Matrix (Normalized)', fontweight='bold')
    axes[1].set_xlabel('Predicted')
    axes[1].set_ylabel('Actual')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    return save_path

# Crear y guardar
with mlflow.start_run(run_id=run_id):
    plot_path = plot_confusion_matrix(y_test, y_pred)
    mlflow.log_artifact(plot_path, artifact_path='evaluation')
    
    print('✓ Confusion matrix guardada')

## 8. Visualización 4: ROC Curve y Precision-Recall

In [ ]:
def plot_roc_and_pr_curves(y_true, y_pred_proba, save_path='roc_pr_curves.png'):
    """Curvas ROC y Precision-Recall"""
    
    # Calcular curvas
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
    pr_auc = auc(recall, precision)
    
    # Crear figura
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # ROC Curve
    axes[0].plot(fpr, tpr, color='darkorange', lw=2,
                label=f'ROC curve (AUC = {roc_auc:.3f})')
    axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
    axes[0].set_xlim([0.0, 1.0])
    axes[0].set_ylim([0.0, 1.05])
    axes[0].set_xlabel('False Positive Rate', fontsize=12)
    axes[0].set_ylabel('True Positive Rate', fontsize=12)
    axes[0].set_title('ROC Curve', fontweight='bold', fontsize=14)
    axes[0].legend(loc='lower right')
    axes[0].grid(True, alpha=0.3)
    
    # Precision-Recall Curve
    axes[1].plot(recall, precision, color='green', lw=2,
                label=f'PR curve (AUC = {pr_auc:.3f})')
    axes[1].set_xlim([0.0, 1.0])
    axes[1].set_ylim([0.0, 1.05])
    axes[1].set_xlabel('Recall', fontsize=12)
    axes[1].set_ylabel('Precision', fontsize=12)
    axes[1].set_title('Precision-Recall Curve', fontweight='bold', fontsize=14)
    axes[1].legend(loc='lower left')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    return save_path, roc_auc, pr_auc

# Crear y guardar
with mlflow.start_run(run_id=run_id):
    plot_path, roc_auc, pr_auc = plot_roc_and_pr_curves(y_test, y_pred_proba)
    
    mlflow.log_artifact(plot_path, artifact_path='evaluation')
    mlflow.log_metrics({
        'roc_auc': roc_auc,
        'pr_auc': pr_auc
    })
    
    print(f'✓ ROC AUC: {roc_auc:.4f}')
    print(f'✓ PR AUC: {pr_auc:.4f}')

## 9. Visualización 5: Feature Importance

In [ ]:
def plot_feature_importance(model, feature_names, top_n=15, save_path='feature_importance.png'):
    """Visualizar importancia de features"""
    
    # Obtener importancias
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1][:top_n]
    
    # Crear DataFrame
    importance_df = pd.DataFrame({
        'feature': [feature_names[i] for i in indices],
        'importance': importances[indices]
    })
    
    # Crear figura
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Barplot horizontal
    sns.barplot(
        data=importance_df,
        y='feature',
        x='importance',
        palette='viridis',
        ax=axes[0]
    )
    axes[0].set_title(f'Top {top_n} Feature Importances', fontweight='bold', fontsize=14)
    axes[0].set_xlabel('Importance', fontsize=12)
    axes[0].set_ylabel('Feature', fontsize=12)
    axes[0].grid(True, alpha=0.3, axis='x')
    
    # Cumulative importance
    cumulative_importance = np.cumsum(importances[indices])
    axes[1].plot(range(1, top_n + 1), cumulative_importance, 
                marker='o', linewidth=2, markersize=8, color='darkblue')
    axes[1].axhline(y=0.95, color='red', linestyle='--', label='95% threshold')
    axes[1].set_title('Cumulative Feature Importance', fontweight='bold', fontsize=14)
    axes[1].set_xlabel('Number of Features', fontsize=12)
    axes[1].set_ylabel('Cumulative Importance', fontsize=12)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    return save_path, importance_df

# Crear y guardar
with mlflow.start_run(run_id=run_id):
    plot_path, importance_df = plot_feature_importance(model, feature_names)
    
    mlflow.log_artifact(plot_path, artifact_path='analysis')
    
    # Guardar CSV de importancias
    csv_path = 'feature_importance.csv'
    importance_df.to_csv(csv_path, index=False)
    mlflow.log_artifact(csv_path, artifact_path='analysis')
    
    # Log top feature
    mlflow.log_param('top_feature', importance_df.iloc[0]['feature'])
    mlflow.log_metric('top_feature_importance', importance_df.iloc[0]['importance'])
    
    print('✓ Feature importance guardada')

## 10. Visualización 6: Comparación de Modelos

In [ ]:
def compare_models_visualization(models_results, save_path='models_comparison.png'):
    """Comparar múltiples modelos visualmente"""
    
    # Crear DataFrame
    df_results = pd.DataFrame(models_results)
    
    # Crear figura
    fig = plt.figure(figsize=(16, 10))
    gs = gridspec.GridSpec(2, 2, figure=fig)
    
    # 1. Accuracy comparison
    ax1 = fig.add_subplot(gs[0, 0])
    sns.barplot(data=df_results, x='model', y='accuracy', palette='Set2', ax=ax1)
    ax1.set_title('Accuracy por Modelo', fontweight='bold', fontsize=12)
    ax1.set_ylim([df_results['accuracy'].min() - 0.05, 1.0])
    ax1.grid(True, alpha=0.3, axis='y')
    
    # 2. F1-Score comparison
    ax2 = fig.add_subplot(gs[0, 1])
    sns.barplot(data=df_results, x='model', y='f1_score', palette='Set3', ax=ax2)
    ax2.set_title('F1-Score por Modelo', fontweight='bold', fontsize=12)
    ax2.set_ylim([df_results['f1_score'].min() - 0.05, 1.0])
    ax2.grid(True, alpha=0.3, axis='y')
    
    # 3. Training time
    ax3 = fig.add_subplot(gs[1, 0])
    sns.barplot(data=df_results, x='model', y='train_time', palette='coolwarm', ax=ax3)
    ax3.set_title('Tiempo de Entrenamiento (s)', fontweight='bold', fontsize=12)
    ax3.grid(True, alpha=0.3, axis='y')
    
    # 4. Radar chart de métricas
    ax4 = fig.add_subplot(gs[1, 1], projection='polar')
    
    metrics = ['accuracy', 'f1_score', 'precision', 'recall']
    angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
    angles += angles[:1]
    
    for idx, row in df_results.iterrows():
        values = [row[m] for m in metrics]
        values += values[:1]
        ax4.plot(angles, values, 'o-', linewidth=2, label=row['model'])
        ax4.fill(angles, values, alpha=0.15)
    
    ax4.set_xticks(angles[:-1])
    ax4.set_xticklabels(metrics)
    ax4.set_ylim(0, 1)
    ax4.set_title('Radar de Métricas', fontweight='bold', fontsize=12, pad=20)
    ax4.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    ax4.grid(True)
    
    plt.suptitle('Comparación de Modelos', fontsize=16, fontweight='bold', y=1.00)
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    return save_path

# Entrenar múltiples modelos
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
}

results = []

for name, model in models.items():
    import time
    
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    y_pred = model.predict(X_test)
    
    from sklearn.metrics import precision_score, recall_score
    
    results.append({
        'model': name,
        'accuracy': accuracy_score(y_test, y_pred),
        'f1_score': f1_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'train_time': train_time
    })

# Visualizar comparación
with mlflow.start_run(run_name='models-comparison'):
    plot_path = compare_models_visualization(results)
    mlflow.log_artifact(plot_path, artifact_path='comparison')
    
    # Log resultados
    for result in results:
        for key, value in result.items():
            if key != 'model':
                mlflow.log_metric(f"{result['model']}_{key}", value)
    
    print('✓ Comparación de modelos guardada')

## 11. Visualización 7: Gráficos Interactivos con Plotly

In [ ]:
def create_interactive_plots(df, y_test, y_pred_proba):
    """Crear visualizaciones interactivas con Plotly"""
    
    # 1. Distribución 3D de features
    fig1 = px.scatter_3d(
        df,
        x='feature_0',
        y='feature_1',
        z='feature_2',
        color='target',
        title='Distribución 3D de Features',
        labels={'target': 'Class'},
        color_continuous_scale='Viridis'
    )
    fig1.write_html('3d_distribution.html')
    
    # 2. ROC Curve interactiva
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=fpr,
        y=tpr,
        mode='lines',
        name=f'ROC (AUC={roc_auc:.3f})',
        line=dict(color='darkorange', width=3),
        hovertemplate='<b>FPR</b>: %{x:.3f}<br><b>TPR</b>: %{y:.3f}'
    ))
    fig2.add_trace(go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode='lines',
        name='Random',
        line=dict(color='navy', width=2, dash='dash')
    ))
    fig2.update_layout(
        title='Interactive ROC Curve',
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        hovermode='closest'
    )
    fig2.write_html('roc_curve_interactive.html')
    
    # 3. Dashboard de métricas
    fig3 = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Distribution', 'Correlation', 'Feature Importance', 'Predictions'),
        specs=[[{'type': 'histogram'}, {'type': 'heatmap'}],
               [{'type': 'bar'}, {'type': 'scatter'}]]
    )
    
    # Histogram
    for class_label in [0, 1]:
        fig3.add_trace(
            go.Histogram(
                x=df[df['target'] == class_label]['feature_0'],
                name=f'Class {class_label}',
                opacity=0.7
            ),
            row=1, col=1
        )
    
    # Heatmap
    corr = df[feature_names[:5]].corr()
    fig3.add_trace(
        go.Heatmap(
            z=corr.values,
            x=corr.columns,
            y=corr.columns,
            colorscale='RdBu'
        ),
        row=1, col=2
    )
    
    fig3.update_layout(height=800, showlegend=True, title_text='ML Dashboard')
    fig3.write_html('ml_dashboard.html')
    
    return ['3d_distribution.html', 'roc_curve_interactive.html', 'ml_dashboard.html']

# Crear gráficos interactivos
with mlflow.start_run(run_name='interactive-plots'):
    html_files = create_interactive_plots(df, y_test, y_pred_proba)
    
    for html_file in html_files:
        mlflow.log_artifact(html_file, artifact_path='interactive')
    
    print('✓ Gráficos interactivos guardados en MLflow')
    print('  Puedes visualizarlos en MLflow UI -> Artifacts -> interactive/')

## 12. Best Practices: Template de Visualización Completa

In [ ]:
def comprehensive_model_report(model, X_train, X_test, y_train, y_test, 
                               feature_names, experiment_name='complete-report'):
    """Reporte completo de modelo con todas las visualizaciones"""
    
    with mlflow.start_run(run_name=experiment_name) as run:
        # 1. Entrenar modelo
        import time
        start_time = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start_time
        
        # 2. Predicciones
        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        # 3. Métricas
        from sklearn.metrics import (
            accuracy_score, f1_score, precision_score, recall_score,
            roc_auc_score
        )
        
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'roc_auc': roc_auc_score(y_test, y_pred_proba),
            'train_time': train_time
        }
        
        mlflow.log_metrics(metrics)
        
        # 4. Visualizaciones
        plots = {}
        
        # Confusion Matrix
        plots['confusion_matrix'] = plot_confusion_matrix(
            y_test, y_pred, 'report_confusion_matrix.png'
        )
        
        # ROC & PR Curves
        plots['roc_pr'], _, _ = plot_roc_and_pr_curves(
            y_test, y_pred_proba, 'report_roc_pr.png'
        )
        
        # Feature Importance
        if hasattr(model, 'feature_importances_'):
            plots['feature_importance'], _ = plot_feature_importance(
                model, feature_names, save_path='report_feature_importance.png'
            )
        
        # Log all plots
        for plot_name, plot_path in plots.items():
            mlflow.log_artifact(plot_path, artifact_path='report')
        
        # 5. Guardar modelo
        mlflow.sklearn.log_model(model, 'model')
        
        # 6. Crear resumen HTML
        html_report = f"""
        <html>
        <head><title>Model Report</title></head>
        <body>
            <h1>Model Performance Report</h1>
            <h2>Metrics</h2>
            <ul>
                <li>Accuracy: {metrics['accuracy']:.4f}</li>
                <li>F1-Score: {metrics['f1_score']:.4f}</li>
                <li>Precision: {metrics['precision']:.4f}</li>
                <li>Recall: {metrics['recall']:.4f}</li>
                <li>ROC-AUC: {metrics['roc_auc']:.4f}</li>
            </ul>
        </body>
        </html>
        """
        
        with open('model_report.html', 'w') as f:
            f.write(html_report)
        
        mlflow.log_artifact('model_report.html', artifact_path='report')
        
        print('\n' + '='*60)
        print('REPORTE COMPLETO GENERADO')
        print('='*60)
        print(f'Run ID: {run.info.run_id}')
        print(f'\nMétricas:')
        for metric, value in metrics.items():
            print(f'  {metric}: {value:.4f}')
        print(f'\nVisualiza el reporte en MLflow UI:')
        print(f'http://localhost:5000/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}')
        print('='*60)
        
        return run.info.run_id

# Ejecutar reporte completo
model_report = RandomForestClassifier(n_estimators=100, random_state=42)
run_id = comprehensive_model_report(
    model_report, X_train, X_test, y_train, y_test, 
    feature_names, 'comprehensive-ml-report'
)

## 13. Limpiar Archivos Temporales

In [ ]:
import os
import glob

# Listar archivos generados
image_files = glob.glob('*.png')
html_files = glob.glob('*.html')
csv_files = glob.glob('*.csv')

print('Archivos generados:')
print(f'  Imágenes: {len(image_files)}')
print(f'  HTML: {len(html_files)}')
print(f'  CSV: {len(csv_files)}')
print('\nTodos están guardados en MLflow!')
print('\nPara limpiar archivos locales (opcional):")
print('  import os; [os.remove(f) for f in glob.glob("*.png") + glob.glob("*.html")]')

## Resumen y Best Practices

### Tipos de Visualizaciones Creadas:

1. **Exploración de Datos**
   - Distribuciones de features
   - Matrices de correlación
   - Gráficos 3D interactivos

2. **Evaluación de Modelos**
   - Confusion Matrix
   - ROC Curve & PR Curve
   - Métricas comparativas

3. **Interpretabilidad**
   - Feature importance
   - Cumulative importance
   - Análisis de contribución

4. **Comparaciones**
   - Multi-model comparisons
   - Radar charts
   - Dashboards interactivos

### Mejores Prácticas:

✅ **Siempre guardar visualizaciones en MLflow**
```python
mlflow.log_artifact(plot_path, artifact_path='plots')
```

✅ **Usar nombres descriptivos**
```python
save_path = 'confusion_matrix_test_set.png'
```

✅ **Alta resolución para publicación**
```python
plt.savefig(path, dpi=150, bbox_inches='tight')
```

✅ **Combinar estáticas e interactivas**
- Matplotlib/Seaborn: Para reportes PDF
- Plotly: Para exploración interactiva

✅ **Organizar artifacts por categoría**
```python
mlflow.log_artifact(path, artifact_path='evaluation')  # vs 'exploration', 'comparison'
```

### Comandos Útiles:

```python
# Buscar runs con visualizaciones
client = MlflowClient()
runs = client.search_runs(experiment_ids=['1'])

# Descargar artifacts
client.download_artifacts(run_id, 'plots', dst_path='./downloads')

# Listar artifacts
artifacts = client.list_artifacts(run_id)
for artifact in artifacts:
    print(artifact.path)
```

### Recursos:
- [Matplotlib Gallery](https://matplotlib.org/stable/gallery/index.html)
- [Seaborn Examples](https://seaborn.pydata.org/examples/index.html)
- [Plotly Charts](https://plotly.com/python/)
- [MLflow Artifacts](https://mlflow.org/docs/latest/tracking.html#logging-artifacts)